# Day 8: Seven Segment Search

(From [Advent of Code 2021, day 8](https://adventofcode.com/2021/day/8))

In [1]:
# Libraries used:
import pandas as pd
import numpy as np

## Part 1

You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit [seven-segment displays](https://en.wikipedia.org/wiki/Seven-segment_display) in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named `a` through `g`:

```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
```

So, to render a `1`, only segments `c` and `f` would be turned on; the rest would be off. To render a `7`, only segments `a`, `c`, and `f` would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires `a` through `g`, but those wires are connected to segments _randomly_. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits _within_ a display use the same connections, though.)

So, you might know that only signal wires `b` and `g` are turned on, but that doesn't mean _segments_ `b` and `g` are turned on: the only digit that uses two segments is `1`, so it must mean segments `c` and `f` are meant to be on. With just that information, you still can't tell which wire (`b`/`g`) goes to which segment (`c`/`f`). For that, you'll need to collect more information.

For each display, you watch the changing signals for a while, make a note of _all ten unique signal patterns_ you see, and then write down a single _four digit output value_ (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

For example, here is what you might see in a single entry in your notes:

```
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
```

(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)

Each entry consists of ten _unique signal patterns_, a `|` delimiter, and finally the _four digit output value_. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because `7` is the only digit that uses three segments, `dab` in the above example means that to render a `7`, signal lines `d`, `a`, and `b` are on. Because `4` is the only digit that uses four segments, `eafb` means that to render a `4`, signal lines `e`, `a`, `f`, and `b` are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (`cdfeb fcadb cdfeb cdbaf`) use five segments and are more difficult to deduce.

For now, _focus on the easy digits_. Consider this larger example:

```
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce
```

Because the digits `1`, `4`, `7`, and `8` each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting _only digits in the output values_ (the part after `|` on each line), in the above example, there are `26` instances of digits that use a unique number of segments.

_In the output values, how many times do digits `1`, `4`, `7`, or `8` appear?_

### Solution

In [2]:
# Input file
# (Set this to the correct path for you!)
data = pd.read_csv('input.txt', header=None, sep='|', names=['signal_pattern', 'output_value'])

# Peak at input data
data.head()

,signal_pattern,output_value
0,ecfdbg decfba aegd fdcag fagecd gd gcafb efdac...,bgacf afdebc fceda cabfg
1,cfdabeg cda bcgad bedfac aefbgd bfgcad dfbga e...,gdbefac fgabcd dcbefag aedgfb
2,aedbfgc gcaf ebgfca bca edgcfb adbge gebac fbe...,ceafbg acb egbad gfaebc
3,gaeb cafdbg aedfc ecfgdb ab cdaefgb cgbdae bde...,gfacdeb gabdec begcd adbce
4,fcbgea fbedg bc cbefg befgcda cfgdab gbc efdgc...,faecg cb cb fecbga


Our problem statement gives the following information:

>Because the digits `1`, `4`, `7`, and `8` each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits.

Let's visualize the seven-segment displays of those four digits:

```
    1:        4:
   ....      ....
  .    c    b    c
  .    c    b    c
   ....      dddd
  .    f    .    f
  .    f    .    f
   ....      ....

    7:      8:    
   aaaa    aaaa   
  .    c  b    c 
  .    c  b    c
   ....    dddd    
  .    f  e    f 
  .    f  e    f  
   ....    gggg   
```

The number of segments for each digit are as follows:
* `1`: 2 segments
* `4`: 4 segments
* `7`: 3 segments
* `8`: 7 segments

Therefore, we are looking for output values with character lengths 2, 3, 4, and 7.

In [3]:
# Transform output value string into a list of separate elements
data['output_value'] = data['output_value'].apply(lambda x: x.split())

In [4]:
# Select output elements that represent digits 1, 4, 7, and 8
# and save them into a list
target_digits_appearance = []
for row in data.index:
    for output in data['output_value'][row]:
        if len(output) == 2:
            target_digits_appearance.append(1)
        elif len(output) == 3:
            target_digits_appearance.append(7)
        elif len(output) == 4:
            target_digits_appearance.append(4)
        elif len(output) == 7:
            target_digits_appearance.append(8)
        else:
            pass

# Count number of elements in list
# (getting number of appearences of digits 1, 4, 7, and 8)
len(target_digits_appearance)

303

*Answer*: 303